<a href="https://colab.research.google.com/github/tim530712345/AIOT_project/blob/main/Yolo_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 啟用GPU後，查看機器的狀態
!nvidia-smi

## 將 Google Drive 掛載到 Colab 目錄下

掛載 Google Drive 的好處是不用每次都手動上傳或下載檔案，而且還能讓訓練好的模型檔案自動保存到 Google Drive。這樣就不會因為 Colab 中斷後就必須從頭訓練。

In [ ]:
"""
掛載 google drive (若有重要資料不想遺失或是想要抓取自己的資料時)
"""
from google.colab import drive
drive.mount("/drive")

Mounted at /drive


In [ ]:
# 查看你的google drive下面有多少東西
!ls /drive

MyDrive  Shareddrives


# 準備資料集

您可以使用自己的資料夾。

為了示範，這邊使用 [The Oxford-IIIT Pet Dataset](http://www.robots.ox.ac.uk/~vgg/data/pets/) 已經標注好的寵物資料來當作範例。我們會用到裡面的圖片檔以及 `annotations` 中的 `xml` 標註資料。

可以依據自己的需求 先製作好自己的 辨識圖像包 & annotation包

In [ ]:
"""
將你要用的訓練資料 & 標註資料複製到Colab中 (通常還是未解壓縮的一個包)

第一種：
若「沒有」事先上傳到google drive，使用以下指令來獲取你要的資料
→ 使用 !wget <檔案下載網址> 

第二種：
若「已有」事先傳到google drive，用以下指令將其複製到Colab
→ 利用 !cp <google drive中的檔案路徑> <要複製到的目的地路徑>

"""
# 第一種
# !wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
# !wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

# 第二種
!cp /drive/MyDrive/Yolo_Practice_Datasets/pet_images.tar.gz /content/
!cp /drive/MyDrive/Yolo_Practice_Datasets/pet_annotations.tar.gz /content/

In [ ]:
"""
將下載下來的資訊放到一個專屬的資料夾 (建議依據你要偵測物件的類型來做資料夾命名)

將未解壓縮的包解壓縮並放到你的專屬資料夾

步驟：
1. 先創建專屬資料夾
→ !mkdir <創建資料夾路徑>

2. 將你的資料解壓縮到專屬資料夾中
→ !tar -xf <壓縮包名稱> -C <解壓縮目的地路徑>

3. 將標註的檔案文件(*.xml檔案)，移動到專屬資料夾內部的labels資料夾內，以好整理
→ !mv <欲移動檔案所在資料夾的路徑> <移動目的地路徑>

4. 移除掉原有標註文件所解壓縮出來的資料夾，這邊使用強制移除，刪除路徑下的資料夾內容全部刪除 (-fr: force remove)
→ !rm -fr <標註資料包解壓縮出來的文件夾路徑>

"""
# 創建專屬資料夾
!mkdir /content/pet_detection

# 解壓縮資料包 (含訓練資料 & 標註資料)
!tar -xf pet_images.tar.gz -C /content/pet_detection
!tar -xf pet_annotations.tar.gz

# 將標註資料的xmls檔案移到專屬資料夾下面的labels資料夾
!mv annotations/xmls /content/pet_detection/labels

# 移除掉標註資料包解壓縮出來的整個文件夾
!rm -fr annotations

## 定義後來會用到的檔案路徑

路徑分為保存在 Colab 的，每次 Colab 被重置後就會消失。所以只放一些不重要的東西。
保存在 Google Drive 的檔案就是會被保存下來的，即使 Colab 被重置後，也不會消失；我們會把重要的東西存在這，例如設定檔、訓練到一半的模型等。

In [ ]:
"""
定義後來會用到的檔案路徑

過程中，資料會保存在兩個地方

保存在 Colab 端的，每次 Colab 被重置後就會消失，所以只放一些不重要的東西。
保存在 Google Drive 端的檔案不會消失，我們會把重要的東西存在這，例如設定檔、訓練到一半的模型等。
"""

# 剛訓練資料還有標註資料同一層的子資料夾的創建路徑
LOCAL_IMAGES_DIR_PATH = "/content/pet_detection/images"
LOCAL_LABELS_DIR_PATH = "/content/pet_detection/labels"
LOCAL_YOLOS_DIR_PATH = "/content/pet_detection/yolos"
LOCAL_CFG_DIR_PATH = "/content/pet_detection/cfg"

# colab本地端APP資料夾內的子資料夾路徑
GDRIVE_APP_BASE_DIR_PATH = "/content/app"
GDRIVE_WEIGHTS_DIR_PATH = GDRIVE_APP_BASE_DIR_PATH+"/weights"
GDRIVE_CFG_DIR_PATH = GDRIVE_APP_BASE_DIR_PATH+"/cfg"

# colab本地端直接存放路徑 (最後訓練完的darknet檔案存放的路徑)
GITHUB_CODEBASE_DIR_PATH = "/content/train_yolo_with_custom_dataset_on_colab"

# google drive裡面的創建的資料夾路徑 (將你訓練完的darknet存放到google drive的路徑  =>  訓練完的模型才不會不見)
GDRIVE_APP_BASE_DIR_REMOTE_PATH = "/drive/MyDrive/train_yolo_with_custom_dataset_on_colab"

GDRIVE_DARKNET_BIN_FILE_PATH = GITHUB_CODEBASE_DIR_PATH+"/darknet"


In [ ]:
"""
暫時不知道幹嘛
或許可參考 https://www.ibm.com/docs/zh-tw/aix/7.1?topic=links-linking-files-ln-command
"""
# build the link to avoiding type in long path name everytime
!ln -fs {GDRIVE_APP_BASE_DIR_REMOTE_PATH} {GDRIVE_APP_BASE_DIR_PATH}

In [ ]:
"""
清空之前建立的一些路徑的資料夾內容
並依照前面製造的路徑，創出資料夾

運用python內建的shutil模組功能 https://docs.python.org/zh-tw/3/library/shutil.html#shutil.rmtree
shutil.rmtree(要刪除的資料夾路徑, 是否忽略刪除過程中的錯誤=True or False)

運用python內建的os模組功能 https://docs.python.org/zh-tw/3/library/os.html#os.makedirs
os.makedirs(欲建立的資料夾路徑, 使否無視已存在之目標資料夾=True or False)

"""

import os
import shutil

shutil.rmtree(LOCAL_CFG_DIR_PATH, ignore_errors=True)
shutil.rmtree(LOCAL_YOLOS_DIR_PATH, ignore_errors=True)

os.makedirs(GDRIVE_APP_BASE_DIR_REMOTE_PATH.replace("\ ", " "), exist_ok=True)
os.makedirs(GDRIVE_CFG_DIR_PATH, exist_ok=True)
os.makedirs(GDRIVE_WEIGHTS_DIR_PATH, exist_ok=True)

os.makedirs(LOCAL_CFG_DIR_PATH, exist_ok=True)
os.makedirs(LOCAL_YOLOS_DIR_PATH, exist_ok=True)
# os.makedirs(GDRIVE_CFG_DIR_PATH, exist_ok=True)

## 擷取出所有的標籤名稱

由於 darknet 框架會將物體名字全部轉成數字，我們需要先將物體名字全部擷取出來存在一份檔案中，當作之後的對照表。

In [ ]:
"""
製作辨識結果對照表

擷取出所有的標籤名稱
由於 Darknet 框架會將物體名字全部轉成數字，我們需要先將物體名字全部擷取出來存在一份檔案中，當作之後的對照表。

使用python內建的glob模組 https://docs.python.org/zh-tw/3/library/glob.html#glob.glob
glob模組的主要方法就是glob, 用來找尋你所需要的文件內容路徑  =>  返回所需文件路徑的LIST
glob.glob(搜尋的路徑, 搜尋的副檔名類型名稱)

正則表達式的使用 => re.findall()  詳細參閱：https://docs.python.org/zh-cn/3/library/re.html#re.findall

關於路徑的合併 https://docs.python.org/zh-tw/3/library/os.path.html#os.path.join

"""
# Convert VOC xmls into Yolo's format

import glob
import os
import re

# 先創一個set用來存放等下抓出來的標註標籤 
labels = set()

# 找到所有的VOC xmls標註檔案路徑，然後打開讀取
for path in glob.glob(os.path.join(LOCAL_LABELS_DIR_PATH, "*.xml")):
    with open(path, "r") as f:
        content = f.read()

    # 抓取每個VOC xmls標註檔案裡面的標註標籤名
    matches = re.findall(r"<name>([\w_]+)<\/name>", content, flags=0)

    #　將抓取出來的標註標籤名加到剛剛創建的set中 (注意，set需使用update，而非append)
    labels.update(matches)

# 將剛剛的標註標籤檔案寫入到 *.names 檔案中 (標註標籤名稱)
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "obj.names"), "w") as f:
    # 將set寫入 *.names 檔案中
    f.write("\n".join(labels))

# 印出來看看到底有幾種標註
print("Read in {} labels: {}".format(len(labels), ",".join(labels)))

Read in 2 labels: dog,cat


## 將 VOC 格式的標記資料轉成 YOLO 的標記資料

Yolo 不是使用標準的格式，原本的 VOC 標記格式需要轉換後才能使用在 darkent 框架上。
這邊就不詳細解釋如何轉換，對如何轉換的詳細規格可以參考 Yolo 官網 。我們直接使用我從 convert2Yolo 套件中擷取出來的片段程式碼來執行轉換，並把轉換的結果都放到 `/content/pet_detection/yolos` 目錄中。

In [ ]:
"""
將 VOC 格式的標註標籤名稱轉成 YOLO 的標註標籤名稱
Yolo 不是使用標準的格式，原本的 VOC 標記格式需要轉換後才能使用在 Darkent 框架上。
這邊就不詳細解釋如何轉換，對如何轉換的詳細規格可以參考 Yolo 官網 。


直接使用從 convert2Yolo 套件中擷取出來的片段程式碼來執行轉換，並把轉換的結果都放到 /content/pet_detection/yolos 目錄中。
"""
# 搞定convert2Yolo套件安裝
!git clone https://github.com/ssaru/convert2Yolo.git
!pip install cycler

Cloning into 'convert2Yolo'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 215 (delta 4), reused 6 (delta 4), pack-reused 201
Receiving objects: 100% (215/215), 1002.72 KiB | 4.82 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [ ]:
import sys
sys.path.append(GITHUB_CODEBASE_DIR_PATH)

# 引用convert2Yolo 
from convert2Yolo.Format import VOC, YOLO

voc = VOC()
yolo = YOLO(os.path.join(GDRIVE_CFG_DIR_PATH, "obj.names"))

# 找到原本所在標註資料的路徑
flag, data = voc.parse(LOCAL_LABELS_DIR_PATH)
#　將data轉換成Yolo格式
flag, data = yolo.generate(data)

# 將轉換成Yolo格式的標註資料儲存起來  =>  path = /content/pet_detection/yolos
flag, data = yolo.save(data,
    save_path=LOCAL_YOLOS_DIR_PATH,
    img_path=LOCAL_IMAGES_DIR_PATH, img_type=".jpg", manifest_path="./")


VOC Parsing:   |████████████████████████████████████████| 100.0% (3686/3686)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (3686/3686)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (3686/3686)  Complete



In [ ]:
"""
將訓練資料複製到Yolo的文件夾中
"""
# 以下幾種寫法都可以
# !find $LOCAL_IMAGES_DIR_PATH -name "*.jpg" -exec cp {} /content/pet_detection/yolos \;

# from distutils.dir_util import copy_tree
# copy_tree(LOCAL_IMAGES_DIR_PATH, LOCAL_YOLOS_DIR_PATH)

!cp {LOCAL_IMAGES_DIR_PATH}/*.jpg {LOCAL_YOLOS_DIR_PATH}

## 準備訓練用的設定檔

* `obj.names`：所有的物體標籤名稱，每一行一個。例如此例中只有兩行，分別是 cat dog 。
* `yolov3.cfg`：Darknet 網路的設定檔，描述每一層網路應該要如何建立，以及建立多少 node 等。裡面有些數值需要根據你的訓練資料來個別設定。
* `train.txt` `test.txt` ：這兩個檔案告訴 Darknet 要到哪個路徑下找到訓練用的圖片。
* `obj.data`：Darknet 的主要設定檔案，告訴 Darknet 其他的設定檔路徑。Darknet 會一一去讀取其他的檔案。

In [ ]:
# 創建屬於自己的cfg檔案  =>  直接從原本的Yolo模型中複製過來再進行後續訓練即可)
# 提供v3 & v4兩種版本，請自行斟酌使用
# !wget https://github.com/pjreddie/darknet/raw/master/cfg/yolov3.cfg
# !mv /content/yolov3.cfg {GDRIVE_CFG_DIR_PATH}
!wget https://github.com/AlexeyAB/darknet/raw/master/cfg/yolov4.cfg
!mv /content/yolov4.cfg {GDRIVE_CFG_DIR_PATH}


# 開啟我們之前轉換成Yolo格式的標註標籤名稱檔案
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "obj.names"), "r") as f:
  f_content = f.read()
label_names = f_content.strip().splitlines()


# *** 準備對原本的Yolo模型cfg檔案做更新，變成我們自己的cfg檔案 *** 

# 開啟複製過來的cfg檔案
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "yolov4.cfg"), "r") as f:
  content = f.read()
# 將所需要的資料寫入到複製的檔案中
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "yolov4.cfg"), "w") as f:
  num_max_batches = len(label_names)*2000
  content = content.replace("%NUM_CLASSES%", str(len(label_names)))
  content = content.replace("%NUM_MAX_BATCHES%", str(num_max_batches))
  content = content.replace("%NUM_MAX_BATCHES_80%", str(int(num_max_batches*0.8)))
  content = content.replace("%NUM_MAX_BATCHES_90%", str(int(num_max_batches*0.9)))
  content = content.replace("%NUM_CONVOLUTIONAL_FILTERS%", str((len(label_names)+5)*3))

  f.write(content)

In [ ]:
"""
產生train & test.txt 兩個標示訓練用圖片路徑的文件檔案
"""
import random
import glob

txt_paths = glob.glob(os.path.join(LOCAL_YOLOS_DIR_PATH, "*.txt"))

random.shuffle(txt_paths)
num_train_images = int(len(txt_paths)*0.8)

assert num_train_images > 0, "There's no training images in folder {}".format(LOCAL_YOLOS_DIR_PATH)

# 在cfg文件夾中創建一個train.txt準備用作寫入
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "train.txt"), 'w') as f:
  for path in txt_paths[:num_train_images]:
    f.write("{}/{}\n".format(LOCAL_YOLOS_DIR_PATH, os.path.basename(path).replace(".txt", ".jpg")))

# 在cfg文件夾中創建一個test.txt準備用作寫入
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "test.txt"), 'w') as f:
  for path in txt_paths[num_train_images:]:
    f.write("{}/{}\n".format(LOCAL_YOLOS_DIR_PATH, os.path.basename(path).replace(".txt", ".jpg")))

# 創建 obj.data 檔案
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "obj.data"), 'w') as f:
  f.write("classes={}\n".format(len(label_names)))
  f.write("train={}/train.txt\n".format(GDRIVE_CFG_DIR_PATH))
  f.write("valid={}/test.txt\n".format(GDRIVE_CFG_DIR_PATH))
  f.write("names={}/obj.names\n".format(GDRIVE_CFG_DIR_PATH))
  f.write("backup={}\n".format(GDRIVE_WEIGHTS_DIR_PATH))

## 準備 darkent 執行檔

我們直接從之前已經編譯好的檔案複製過來就好，不用每次都重頭編譯那實在是太~花~時~間~了~。編譯的方法請見 「[如何在 Colab 安裝 Darknet 框架訓練 YOLO v3 物件辨識並且最佳化 Colab 的訓練流程](http://bit.ly/33XjcEu)」這邊文章。

In [ ]:
"""
clone darknet 到你的colab虛擬機端  (也可直接在)
"""
# 複製Darknet到你的colab
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
# 查看目前colab的CUDA版本
!nvcc --version

In [ ]:
# 安裝支援CUDA版本的cuDNN (將其安裝到cola的 /usr/local 路徑中)
!tar -xzvf /drive/MyDrive/cudnn-11.3-linux-aarch64sbsa-v8.2.1.32.tgz -C /usr/local/

In [ ]:
# 啟用 所有人(all) 可以讀取(read)的權限 a+r
!chmod a+r /usr/local/cuda/include/cudnn.h

In [ ]:
# 檢查cuDNN是否順利安裝完成 (應為空白)
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

In [ ]:
# 到複製的darknet路徑下，然後修改Makefile檔案裡面的值，以啟用GPU來訓練模型
"""
方法一：
使用sed "s/原本字符/替代字符/第幾次出現的位置做取代' <執行此功能的檔案名稱>
指令 s 是取代，把 have 這個字取代為 had，旗標 /1 指的是在第一次出現的位置。如果是 /2 則是第二次出現的位置，以此類推

方法二：
直接去打開darknet資料夾底下的Makefile修改GPU、CUDNN、OPENCV的值
"""
%cd ./darknet

直接去打開darknet資料夾底下的Makefile修改GPU、CUDNN、OPENCV的值 (弄完再註解掉)


# 編譯darknet的compile檔案
!make


# 最後記得切回原本路徑
%cd /content

In [ ]:
# 可將編譯完成的執行檔複製回 Google Drive (這樣下次開始就可以直接叫出無須再編譯)

# 複製回google drive
# !cp ./darknet /app/darknet

In [ ]:
# 之後若從google drive把之前已經編譯好的Darknet抓下來的話，執行下面代碼
# !cp ./darknet /content/darknet

# 執行
# %cd /content
# !cp /app/darknet ./
# !chmod +x ./darknet

## （可選）使用 darknet 預先訓練的基底模型

Darknet 也好心的提供了預先訓練的模型，以此為基底，可以讓後來的訓練比較快達到較好的辨識率。但前提是你的圖片都是常見的圖片，例如一般照片、場景照片等；如果是一些遊戲畫面很少見的，從 0 開始訓練可能會達到比較好的效果。

In [ ]:
"""
下載 Yolo v3 跟 Yolo v4 事先訓練好的比重資料 到darknet資料夾中 (*.cfg資料 & weights資料)
"""
%cd ./darknet

!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://pjreddie.com/media/files/darknet53.conv.74
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights 
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# 正式訓練模型了

上面的步驟看起來很多，但其實只要寫好一次，之後每次訓練時只要換上自己要的資料夾，然後按 `Run all` 就可以了。

這邊提供兩條指令：

1. 是從 YOLO Pre-trained model 開始訓練，
  * 如果你的資料集是一般照片，那用這個效果會比較好。

2. 是從上次訓練的中斷點繼續往下訓練；指令會讀取資料夾內的 weight 檔案，從上次中斷點繼續訓練下去。
  * 適合從 0 開始訓練或是被 Colab 中斷後重新開始訓練。

有關中斷繼續的訓練可參閱
https://pjreddie.com/darknet/yolo/

In [ ]:
# 從頭開始訓練
# !./darknet detector train {GDRIVE_CFG_DIR_PATH}/obj.data {GDRIVE_CFG_DIR_PATH}/yolov4.cfg darknet53.conv.74 -dont_show

""" 注意，若發生 darknet: ./src/utils.c:326: error: Assertion `0‘ failed.錯誤時
到 {GDRIVE_CFG_DIR_PATH}/yolov4.cfg 這個檔案，將subdivisions的值改成64 (依照錯誤提示調整)"""

!./darknet detector train {GDRIVE_CFG_DIR_PATH}/obj.data {GDRIVE_CFG_DIR_PATH}/yolov4.cfg yolov4.conv.137 -dont_show | grep "avg loss"


# 從上次中斷地方訓練
# !./darknet detector train {GDRIVE_CFG_DIR_PATH}/obj.data {GDRIVE_CFG_DIR_PATH}/yolov4.cfg {GDRIVE_WEIGHTS_DIR_PATH}/yolov4_last.weights  -dont_show | grep "avg loss"

串流輸出內容已截斷至最後 5000 行。
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.554583), count: 3, class_loss = 3.722883, iou_loss = 2.020784, total_loss = 5.743668 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.577812), count: 8, class_loss = 8.876036, iou_loss = 2.041097, total_loss = 10.917132 
 total_bbox = 178341, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.000000), count: 1, class_loss = 0.000053, iou_loss = 0.000000, total_loss = 0.000053 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.553546), count: 1, class_loss = 1.185523, iou_loss = 0.636866, total_loss = 1.822389 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.587562), count: 10, class_loss = 8.323679, iou_loss = 2.697896, total_loss = 11.021575 
 total_bbox = 178352, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (io

# 檢視訓練成果

檢測圖片時，我們只需要 `obj.names`, `yolov3.cfg` 以及 weights 檔就夠。我們可以直接利用 Opencv 內建的 darknet 來讀取網路並產生出預測。

In [ ]:
# 預測照片的路徑
TO_DETECTING_IMAGE_DIR_PATH = GITHUB_CODEBASE_DIR_PATH+"/to_detect_images"

# 開啟預測照片
import os
import cv2
import numpy as np
import glob
from google.colab.patches import cv2_imshow
import pprint

pp = pprint.PrettyPrinter(indent=4)


# 寫一個偵測一張照片的功能
def detecting_one_image(net, output_layers, img):
  # Detecting objects
  # cv::dnn::blobFromImage (InputArray image, double scalefactor=1.0, const Size &size=Size(), const Scalar &mean=Scalar(), bool swapRB=false, bool crop=false, int ddepth=CV_32F)
  blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
  net.setInput(blob)
  outs = net.forward(output_layers)

  return outs

# 讀取Yolo Model (依據情況調整weights跟cfg檔案)
net = cv2.dnn.readNet(GDRIVE_WEIGHTS_DIR_PATH+"/yolov4_last.weights", GDRIVE_CFG_DIR_PATH+"/yolov4.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# 讀取轉成Yolo格式的標註物件名稱
with open(GDRIVE_CFG_DIR_PATH+"/obj.names", "r") as f:
  classes = [line.strip() for line in f.readlines()]

# Generate display colors
colors = np.random.uniform(0, 255, size=(len(classes), 3))

for fpath in glob.glob(os.path.join(TO_DETECTING_IMAGE_DIR_PATH, "*.jpg")):
  print("fpath", fpath)

  # 讀取圖片，產生圖片變數
  img = cv2.imread(fpath)
  height, width, channels = img.shape

  # 對辨識圖片做尺寸的重製
  if width>800:
    dim = (800, int(800*height/width))
    img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    height, width, channels = img.shape

  outs = detecting_one_image(net, output_layers, img)

  # 呈現圖像訊息
  for out in outs:
    for detection in out:
      scores = detection[5:]
      class_id = np.argmax(scores)
      confidence = scores[class_id]

      # 當信心程度大於設定值才偵測物件
      if confidence > 0.3:
        center_x = int(detection[0] * width)
        center_y = int(detection[1] * height)
        w = int(detection[2] * width)
        h = int(detection[3] * height)

        # 畫辨識框框
        x = int(center_x - w / 2)
        y = int(center_y - h / 2)

        label = "{} {}".format(round(confidence,2), classes[class_id])

        cv2.rectangle(img, (x, y), (x + w, y + h), colors[class_id], 2)

        # 將辨識的標籤也呈現在照片上
        cv2.putText(img, label, (x, y+h-5), cv2.FONT_HERSHEY_PLAIN, 1, colors[class_id], 1)

  cv2_imshow(img)